In [1]:
from datasets import load_dataset

dataset = load_dataset("PiC/phrase_similarity")

In [2]:
from gensim.scripts.glove2word2vec import glove2word2vec

glove_input_file = r"C:\Users\sevan\Desktop\IIIT-H\Phrase and Sentence\glove.6B\glove.6B.50d.txt" # Adjust the file name/path as necessary
word2vec_output_file = r'glove.6B.100d.word2vec.txt'
glove2word2vec(glove_input_file, word2vec_output_file)


C:\Users\sevan\AppData\Local\Temp/ipykernel_37112/3870872605.py:5: DeprecationWarning: Call to deprecated `glove2word2vec` (KeyedVectors.load_word2vec_format(.., binary=False, no_header=True) loads GLoVE text vectors.).
  glove2word2vec(glove_input_file, word2vec_output_file)


(400000, 50)

In [3]:
from gensim.models import KeyedVectors


filename = 'glove.6B.100d.word2vec.txt'
model = KeyedVectors.load_word2vec_format(filename, binary=False)


vector = model['computer']  
print(vector)


[ 0.079084 -0.81504   1.7901    0.91653   0.10797  -0.55628  -0.84427
 -1.4951    0.13418   0.63627   0.35146   0.25813  -0.55029   0.51056
  0.37409   0.12092  -1.6166    0.83653   0.14202  -0.52348   0.73453
  0.12207  -0.49079   0.32533   0.45306  -1.585    -0.63848  -1.0053
  0.10454  -0.42984   3.181    -0.62187   0.16819  -1.0139    0.064058
  0.57844  -0.4556    0.73783   0.37203  -0.57722   0.66441   0.055129
  0.037891  1.3275    0.30991   0.50697   1.2357    0.1274   -0.11434
  0.20709 ]


In [4]:
import numpy as np
import pandas as pd
from gensim.models import KeyedVectors
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression


def load_glove_model(glove_file):
    model = KeyedVectors.load_word2vec_format(glove_file, binary=False)
    return model

# Function to preprocess and vectorize a phrase
def phrase_to_vec(phrase, model):
    words = phrase.lower().split()
    vectors = [model[word] for word in words if word in model]
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(model.vector_size)

# Compute cosine similarity between two vectors
def cosine_similarity(vec1, vec2):
    if np.all(vec1 == 0) or np.all(vec2 == 0):
        return 0
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

# Load embeddings
glove_model = load_glove_model('glove.6B.100d.word2vec.txt')


data = pd.read_parquet(r"C:\Users\sevan\Desktop\IIIT-H\Phrase and Sentence\0000.parquet")  

# Vectorize phrases
data['vec1'] = data['phrase1'].apply(lambda x: phrase_to_vec(x, glove_model))
data['vec2'] = data['phrase2'].apply(lambda x: phrase_to_vec(x, glove_model))

# Calculate similarities
data['similarity'] = data.apply(lambda row: cosine_similarity(row['vec1'], row['vec2']), axis=1)

#classification threshold
threshold = 0.5
data['predicted_label'] = (data['similarity'] > threshold).astype(int)

# Evaluation
X_train, X_test, y_train, y_test = train_test_split(data['similarity'], data['label'], test_size=0.25, random_state=42)
clf = LogisticRegression()
clf.fit(X_train.values.reshape(-1, 1), y_train)
predictions = clf.predict(X_test.values.reshape(-1, 1))

print("Accuracy:", accuracy_score(y_test, predictions))
print("Classification Report:\n", classification_report(y_test, predictions))


Accuracy: 0.5071387778412336
Classification Report:
               precision    recall  f1-score   support

           0       0.50      0.55      0.52       864
           1       0.51      0.47      0.49       887

    accuracy                           0.51      1751
   macro avg       0.51      0.51      0.51      1751
weighted avg       0.51      0.51      0.51      1751



In [16]:
import numpy as np
from gensim.models import KeyedVectors


def load_glove_model(glove_file):
    model = KeyedVectors.load_word2vec_format(glove_file, binary=False)
    return model

model = load_glove_model('glove.6B.100d.word2vec.txt') 

# Function to convert a phrase to a vector
def phrase_to_vec(phrase, model):
    words = phrase.lower().split()
    vectors = [model[word] for word in words if word in model]
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(model.vector_size)

# Compute cosine similarity between two vectors
def cosine_similarity(vec1, vec2):
    if np.all(vec1 == 0) or np.all(vec2 == 0):
        return 0
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

# Function to determine similarity
def are_phrases_similar(phrase1, phrase2, model, threshold=0.5):
    vec1 = phrase_to_vec(phrase1, model)
    vec2 = phrase_to_vec(phrase2, model)
    similarity = cosine_similarity(vec1, vec2)
    return similarity > threshold, similarity

# Example phrases
phrase1 = "tasty fruit"
phrase2 = "disciplined student"

# Check similarity
similar, similarity_score = are_phrases_similar(phrase1, phrase2, model)
print(f"Are the phrases similar? {'Yes' if similar else 'No'} (Similarity Score: {similarity_score:.4f})")


Are the phrases similar? No (Similarity Score: 0.0366)


In [1]:
import numpy as np
from gensim.models import KeyedVectors

# Load the GloVe model
def load_glove_model(glove_file):
    model = KeyedVectors.load_word2vec_format(glove_file, binary=False)
    return model

model = load_glove_model('glove.6B.100d.word2vec.txt')  

# Function to convert a phrase to a vector
def phrase_to_vec(phrase, model):
    words = phrase.lower().split()
    vectors = [model[word] for word in words if word in model]
    if vectors:
        print(vectors)
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(model.vector_size)

# Compute cosine similarity between two vectors
def cosine_similarity(vec1, vec2):
    if np.all(vec1 == 0) or np.all(vec2 == 0):
        return 0
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))


def are_phrases_similar(phrase1, phrase2, model, threshold=0.5):
    vec1 = phrase_to_vec(phrase1, model)
    vec2 = phrase_to_vec(phrase2, model)
    similarity = cosine_similarity(vec1, vec2)
    return similarity > threshold, similarity

# Example phrases
phrase1 = "There was an unusual period of inactivity across much of the basin from January to March, typically the most active months."
phrase2 = "There was an atypical era of inactivity across much of the basin from January to March, typically the most active months."

# Check similarity
similar, similarity_score = are_phrases_similar(phrase1, phrase2, model)
print(f"Are the phrases similar? {'Yes' if similar else 'No'} (Similarity Score: {similarity_score:.4f})")


[array([ 0.68491 ,  0.32385 , -0.11592 , -0.35925 ,  0.49889 ,  0.042541,
       -0.40153 , -0.36793 , -0.61441 , -0.41148 , -0.3482  , -0.21952 ,
       -0.22393 , -0.64966 ,  0.85443 ,  0.33582 ,  0.2931  ,  0.16552 ,
       -0.55082 , -0.61277 , -0.14768 ,  0.47551 ,  0.65877 , -0.07103 ,
        0.56147 , -1.2651  , -0.74117 ,  0.36365 ,  0.5623  , -0.27365 ,
        3.8506  ,  0.27645 , -0.1009  , -0.71568 ,  0.18511 , -0.12312 ,
        0.56631 , -0.22377 , -0.016831,  0.57539 , -0.51761 ,  0.033823,
        0.19643 ,  0.63498 , -0.24866 ,  0.038716, -0.50559 ,  0.17874 ,
       -0.1693  ,  0.062375], dtype=float32), array([ 0.086888, -0.19416 , -0.24267 , -0.33391 ,  0.56731 ,  0.39783 ,
       -0.97809 ,  0.03159 , -0.61469 , -0.31406 ,  0.56145 ,  0.12886 ,
       -0.84193 , -0.46992 ,  0.47097 ,  0.023012, -0.59609 ,  0.22291 ,
       -1.1614  ,  0.3865  ,  0.067412,  0.44883 ,  0.17394 , -0.53574 ,
        0.17909 , -2.1647  , -0.12827 ,  0.29036 , -0.15061 ,  0.35242 ,
    